import libraries

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from functools import reduce

define hyperparameters

In [2]:
EPOCH = 1000
BATCH_SIZE = 64
LR = 0.01

download and import mnist data

In [3]:
train_data = torchvision.datasets.MNIST(root='./',train=False,download=True,transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root='./',train=False,transform=torchvision.transforms.ToTensor())
train_loader = Data.DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_x = Variable(test_data.test_data.view(-1, 28*28)).type(torch.FloatTensor)
test_y = test_data.test_labels

Define Fully Connected Neural Network

In [4]:
class FCNets(nn.Module):
    def __init__(self,nlist):
        super(FCNets,self).__init__()
        if len(nlist) < 2:
            print('error:not enough layers')
        else:
            self.fc = nn.Sequential()
            for n in range(len(nlist)-1):
                self.fc.add_module('linear' + str(n+1), nn.Linear(in_features=nlist[n], out_features=nlist[n+1]))
                self.fc.add_module('relu' + str(n+1), nn.ReLU())
            #self.fc.add_module('softmax', nn.Softmax())
    def forward(self,x):
        return self.fc(x)

Define Neural Net instance and optimizer, loss function

In [5]:
fc = FCNets([28*28,32,10])
print(fc)
optimizer = torch.optim.SGD(params=fc.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

FCNets(
  (fc): Sequential(
    (linear1): Linear(in_features=784, out_features=32)
    (relu1): ReLU()
    (linear2): Linear(in_features=32, out_features=10)
    (relu2): ReLU()
  )
)


In [6]:
for epoch in range(EPOCH):
    for step,(x,y) in enumerate(train_loader):
        x = x.view(-1,28*28)
        bx = Variable(x)
        by = Variable(y)
        output = fc(bx)
        loss = loss_func(output, by)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if epoch%5 == 0:
        y_predict = torch.max(fc(test_x),1)[1].data
        accuracy = sum(y_predict == test_y)/len(test_y)
        print('Epoch:',epoch,'train loss:%.4f' %loss.data[0],'test accuracy is:%.3f' %accuracy)

Epoch: 0 train loss:2.1022 test accuracy is:0.464
Epoch: 5 train loss:0.5180 test accuracy is:0.855
Epoch: 10 train loss:0.7293 test accuracy is:0.887
Epoch: 15 train loss:0.1292 test accuracy is:0.900
Epoch: 20 train loss:0.3315 test accuracy is:0.909
Epoch: 25 train loss:0.1575 test accuracy is:0.913
Epoch: 30 train loss:0.0921 test accuracy is:0.917
Epoch: 35 train loss:0.1337 test accuracy is:0.921
Epoch: 40 train loss:0.3854 test accuracy is:0.917
Epoch: 45 train loss:0.0991 test accuracy is:0.922
Epoch: 50 train loss:0.1218 test accuracy is:0.921
Epoch: 55 train loss:0.0917 test accuracy is:0.923
Epoch: 60 train loss:0.0623 test accuracy is:0.926
Epoch: 65 train loss:0.1326 test accuracy is:0.924
Epoch: 70 train loss:0.0731 test accuracy is:0.927
Epoch: 75 train loss:0.3682 test accuracy is:0.922
Epoch: 80 train loss:0.1956 test accuracy is:0.930
Epoch: 85 train loss:0.0486 test accuracy is:0.926
Epoch: 90 train loss:0.0683 test accuracy is:0.931
Epoch: 95 train loss:0.2008 test 

if neural nets size is 784 10
with lr = 0.01 
achieve accuracy of 92%
if neural nets size is 784 32 10
with lr = 0.01 epoch=1000
achieve accuracy of 98.7%

In [7]:
for param in fc.parameters():
    print(type(param.data), param.size())

<class 'torch.FloatTensor'> torch.Size([32, 784])
<class 'torch.FloatTensor'> torch.Size([32])
<class 'torch.FloatTensor'> torch.Size([10, 32])
<class 'torch.FloatTensor'> torch.Size([10])


In [8]:
torch.save(fc,'fc.pkl')

/home/yunyou/anaconda3/lib/python3.6/site-packages/torch/serialization.py:158: UserWarning: Couldn't retrieve source code for container of type FCNets. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
